# U-net
Based on https://www.kaggle.com/phoenigs/u-net-dropout-augmentation-stratification

In [ ]:
import numpy as np
import pandas as pd
import cv2

from random import randint

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage import feature, filters
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates

from keras.preprocessing.image import load_img
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, RepeatVector, Reshape, BatchNormalization, UpSampling2D, Concatenate
#from keras.layers.normalization import BatchNormalization

from tqdm import tqdm_notebook
from ipywidgets import IntProgress

import warnings
warnings.simplefilter("ignore", UserWarning)

## Params and helpers

In [ ]:
img_size_ori = 101
img_size_target = 128
n_features = 1

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

def weighted_bce_loss(y_true, y_pred, weight):
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    loss = weight * (logit_y_pred * (1. - y_true) + 
                     K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd
    averaged_mask = K.pool2d(
            y_true, pool_size=(50, 50), strides=(1, 1), padding='same', pool_mode='avg')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight = 5. * K.exp(-5. * K.abs(averaged_mask - 0.5))
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + dice_loss(y_true, y_pred)
    return loss

## Loading of training/testing ids and depths

In [ ]:
train_df = pd.read_csv("../../tgs_data/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../../tgs_data/depths.csv", index_col="id")

# standardize depths
depths_mean = np.mean(depths_df['z'].values)
depths_std = np.std(depths_df['z'].values)
depths_df['z'] = (depths_df['z'] - depths_mean) / depths_std

train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

## Load the images and masks into the DataFrame and divide the pixel values by 255
If run into "IntProgress not defined" error, run jupyter nbextension enable --py --sys-prefix widgetsnbextension in command line.

In [ ]:
train_df["images"] = [np.array(load_img("../../tgs_data/train/images/{}.png".format(idx), grayscale=True)) / 
                      255 for idx in tqdm_notebook(train_df.index)];

In [ ]:
train_df["masks"] = [np.array(load_img("../../tgs_data/train/masks/{}.png".format(idx), grayscale=True)) / 
                     255 for idx in tqdm_notebook(train_df.index)];

In [ ]:
#test_df["images"] = [np.array(load_img("../../tgs_data/test/{}.png".format(idx), grayscale=True)) / 
#                     255 for idx in tqdm_notebook(test_df.index)];

## Calculating the salt coverage and salt coverage classes

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

## Create train/validation split stratified by salt coverage

In [ ]:
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_valid, depth_train, depth_valid = train_test_split(
    train_df.index.values,
    np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.3, stratify=train_df.coverage_class, random_state=1337);

In [ ]:
# create extra features
#x_feat_train = depth_train.T
#x_feat_valid = depth_valid.T

In [ ]:
idx = 1

tmp_img = np.zeros((img_size_target, img_size_target), dtype=train_df.images.loc[ids_train[idx]].dtype)
tmp_img[:img_size_ori, :img_size_ori] = train_df.images.loc[ids_train[idx]]

fix, axs = plt.subplots(1, 2, figsize=(15,5))
axs[0].imshow(tmp_img, cmap="Greys")
axs[0].set_title("Original image")
axs[1].imshow(x_train[idx].squeeze(), cmap="Greys")
axs[1].set_title("Scaled image")

## Data augmentation

In [ ]:
# For training set
x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
x_train = np.append(x_train, [np.rot90(x) for x in x_train], axis=0)
y_train = np.append(y_train, [np.fliplr(y) for y in y_train], axis=0)
y_train = np.append(y_train, [np.rot90(y) for y in y_train], axis=0)

# Copy the depth 3 more times as well
#x_feat_train = np.repeat(x_feat_train, 4, axis=0)

In [ ]:
# Function to distort image
def elastic_transform(image, alpha, sigma, alpha_affine, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_ (with modifications).
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
         Convolutional Neural Networks applied to Visual Document Analysis", in
         Proc. of the International Conference on Document Analysis and
         Recognition, 2003.

     Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    
    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0] + square_size, center_square[1] - square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

In [ ]:
# Elastic transformation
#for i in range(x_train.shape[0]):
#    im = x_train[i][:, :, 0]
#    im_mask = y_train[i] [:, :, 0]   
#    im_merge = np.concatenate((im[...,None], im_mask[...,None]), axis=2)
    
#    # Apply transformation on image
#    im_merge_t = elastic_transform(im_merge, im_merge.shape[1], im_merge.shape[1] * 0.07, im_merge.shape[1] * 0.09)

#    # Split image and mask
#    im_t = im_merge_t[...,0]
#    im_mask_t = im_merge_t[...,1]

#    #plt.figure(figsize = (16,14))
#    #plt.imshow(np.c_[np.r_[im, im_mask], np.r_[im_t, im_mask_t]], cmap='gray')

#    x_train = np.append(x_train, im_t.reshape(img_size_target, img_size_target, 1))
#    y_train = np.append(y_train, im_mask_t.reshape(img_size_target, img_size_target, 1))

In [ ]:
# Add edge detection channel
new_x_train = np.zeros((len(x_train), img_size_target, img_size_target, 3))

for i in range(x_train.shape[0]):
    img = x_train[i, :, :, 0]
    edges1 = filters.sobel(img)
    edges2 = feature.canny(img, sigma=1.5)
    new_x_train[i] = np.concatenate((img.reshape(img_size_target, img_size_target, 1), 
                                     edges1.reshape(img_size_target, img_size_target, 1),
                                     edges2.reshape(img_size_target, img_size_target, 1)), axis=2)
    
x_train = new_x_train  # there got to be a better way

In [ ]:
new_x_valid = np.zeros((len(x_valid), img_size_target, img_size_target, 3))

for i in range(x_valid.shape[0]):
    img = x_valid[i, :, :, 0]
    edges1 = filters.sobel(img)
    edges2 = feature.canny(img, sigma=1.5)
    new_x_valid[i] = np.concatenate((img.reshape(img_size_target, img_size_target, 1), 
                                     edges1.reshape(img_size_target, img_size_target, 1),
                                     edges2.reshape(img_size_target, img_size_target, 1)), axis=2)
    
x_valid = new_x_valid  # there got to be a better way

## Build model

In [ ]:
def conv_block(m, dim, acti, bn, res, do=0):
    n = Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block(m, dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block(n, dim, acti, bn, res)
    else:
        m = conv_block(m, dim, acti, bn, res, do)
    return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
         dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
    i = Input(shape=img_shape)
    o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    o = Conv2D(out_ch, 1, activation='sigmoid')(o)
    return Model(inputs=i, outputs=o)

In [ ]:
model = UNet((img_size_target, img_size_target, 3), start_ch=16, depth=5, batchnorm=True)
model.compile(loss=bce_dice_loss, optimizer="adam", metrics=["accuracy"])
model.summary()

In [ ]:
## def build_model(input_layer, input_features, start_neurons):
#def build_model(input_layer, start_neurons):
#    # 128 -> 64
#    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
#    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
#    #conv1 = BatchNormalization()(conv1)
#    pool1 = MaxPooling2D((2, 2))(conv1)
#    pool1 = Dropout(0.25)(pool1)

#    # 64 -> 32
#    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
#    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
#    #conv2 = BatchNormalization()(conv2)
#    pool2 = MaxPooling2D((2, 2))(conv2)
#    pool2 = Dropout(0.5)(pool2)

#    # 32 -> 16
#    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
#    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
#    #conv3 = BatchNormalization()(conv3)
#    pool3 = MaxPooling2D((2, 2))(conv3)
#    pool3 = Dropout(0.5)(pool3)

#    # 16 -> 8
#    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
#    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
#    #conv4 = BatchNormalization()(conv4)
#    pool4 = MaxPooling2D((2, 2))(conv4)
#    pool4 = Dropout(0.5)(pool4)

#    # Join features information in the deepest layer
#    #f_repeat = RepeatVector(8*8)(input_features)
#    #f_conv = Reshape((8, 8, n_features))(f_repeat)
#    #pool4_feat = concatenate([pool4, f_conv], -1)
    
#    # Middle
#    #convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4_feat)
#    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
#    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)
#    #convm = BatchNormalization()(convm)

#    # 8 -> 16
#    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
#    uconv4 = concatenate([deconv4, conv4])
#    uconv4 = Dropout(0.5)(uconv4)
#    #uconv4 = BatchNormalization()(uconv4)
#    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
#    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

#    # 16 -> 32
#    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
#    uconv3 = concatenate([deconv3, conv3])
#    uconv3 = Dropout(0.5)(uconv3)
#    #uconv3 = BatchNormalization()(uconv3)
#    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
#    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    
#    # 32 -> 64
#    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
#    uconv2 = concatenate([deconv2, conv2])
#    uconv2 = Dropout(0.5)(uconv2)
#    #uconv2 = BatchNormalization()(uconv2)
#    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
#    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    
#    # 64 -> 128
#    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
#    uconv1 = concatenate([deconv1, conv1])
#    uconv1 = Dropout(0.5)(uconv1)
#    #uconv1 = BatchNormalization()(uconv1)
#    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
#    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    
#    uncov1 = Dropout(0.5)(uconv1)
#    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
#    return output_layer

#input_layer = Input((img_size_target, img_size_target, 1))
##input_features = Input((n_features, ), name='feat')
#output_layer = build_model(input_layer, 2)
##output_layer = build_model(input_layer, input_features, 2)

In [ ]:
#model = Model(inputs=input_layer, outputs=output_layer)
##model = Model(inputs=[input_layer, input_features], outputs=output_layer)
#model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#odel.summary()

## Train model

In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000005, verbose=1)

epochs = 200
batch_size = 32

#history = model.fit([x_train, x_feat_train], y_train,
history = model.fit(x_train, y_train,
                    #validation_data=[[x_valid, x_feat_valid], y_valid],
                    validation_data=[x_valid, y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(15,5))
ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
ax_loss.legend()
ax_acc.plot(history.epoch, history.history["acc"], label="Train accuracy")
ax_acc.plot(history.epoch, history.history["val_acc"], label="Validation accuracy")
ax_acc.legend()

In [ ]:
model = load_model("./keras.model")

## Predict the validation set to do a sanity check

In [ ]:
#preds_valid = model.predict([x_valid, x_feat_valid]).reshape(-1, img_size_target, img_size_target)
preds_valid = model.predict(x_valid).reshape(-1, img_size_target, img_size_target)
preds_valid = np.array([downsample(x) for x in preds_valid])
y_valid = np.array([downsample(x) for x in y_valid])

In [ ]:
y_valid_ori = np.array([train_df.loc[idx].masks for idx in ids_valid])

In [ ]:
max_images = 60
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(ids_valid[:max_images]):
    img = train_df.loc[idx].images
    mask = train_df.loc[idx].masks
    pred = preds_valid[i]
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.imshow(pred, alpha=0.3, cmap="OrRd")
    ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
    ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
    ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt, Red: prediction. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")

## Score the model and do a threshold optimization by the best IoU

In [ ]:
## src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)

In [ ]:
thresholds = np.linspace(0, 1, 50)
ious = np.array([iou_metric_batch(y_valid_ori, np.int32(preds_valid > threshold)) for threshold in tqdm_notebook(thresholds)])

In [ ]:
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

In [ ]:
plt.plot(thresholds, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

## Another sanity check with adjusted threshold

In [ ]:
max_images = 60
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(ids_valid[:max_images]):
    img = train_df.loc[idx].images
    mask = train_df.loc[idx].masks
    pred = preds_valid[i]
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.imshow(np.array(np.round(pred > threshold_best), dtype=np.float32), alpha=0.3, cmap="OrRd")
    ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
    ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
    ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt, Red: prediction. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")

## Prepare for submission

In [ ]:
# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

In [ ]:
x_test = np.array([upsample(np.array(load_img("../../tgs_data/test/{}.png".format(idx), grayscale=True))) 
                   / 255 for idx in tqdm_notebook(test_df.index)]).reshape(-1, img_size_target, img_size_target, 1)

In [ ]:
# Add edge detection channel
new_x_test = np.zeros((len(x_test), img_size_target, img_size_target, 3))

for i in range(x_test.shape[0]):
    img = x_test[i, :, :, 0]
    edges1 = filters.sobel(img)
    edges2 = feature.canny(img, sig=1.5)
    new_x_test[i] = np.concatenate((img.reshape(img_size_target, img_size_target, 1), 
                                    edges1.reshape(img_size_target, img_size_target, 1),
                                    edges2.reshape(img_size_target, img_size_target, 1)), axis=2)
    
x_test = new_x_test  # there got to be a better way

In [ ]:
preds_test = model.predict(x_test)
#preds_test = model.predict([x_test, depths_test])

In [ ]:
pred_dict = {idx: RLenc(np.round(downsample(preds_test[i]) > threshold_best)) 
             for i, idx in enumerate(tqdm_notebook(test_df.index.values))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('../submissions/submission_08_20.csv')